# Explore Music  Similarity & Popularity

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from networkx.algorithms import community

## Data Imports

In [2]:
df_track = pd.read_csv('../data/track.csv')
df_album = pd.read_csv('../data/album.csv')
df_artist = pd.read_csv('../data/artist.csv')
df_rating = pd.read_csv('../data/rating.csv')
df_similar = pd.read_csv('../data/track_similarity.csv')
df_weekly_rating = pd.read_csv('../data/weekly_rating.csv')

## Useful Functions

In [3]:
# ======================= Convert Names to IDs =============================

def convert_artist_name_to_id(df_artist, artist_name):
    return df_artist[df_artist['name'] == artist_name]['artist_id'].values[0]

def convert_album_name_to_id(album_name):
    return df_album[df_album['name'] == album_name]['album_id'].values[0]

def convert_track_name_to_id(track_name):
    return df_track[df_track['name'] == track_name]['track_id'].values[0]

# ======================= Convert Ids to Names =============================

def convert_track_id_to_name(track_id):
    return df_track[df_track['track_id'] == track_id]['name'].values[0]

def convert_album_id_to_name(album_id):
    return df_album[df_album['album_id'] == album_id]['name'].values[0]

def convert_artist_id_to_name(artist_id):
    return df_artist[df_artist['artist_id'] == artist_id]['name'].values[0]

# ======================= Others =============================

def get_all_artist_albums(artist_name, byName=False):
    all_albums = df_artist[df_artist['name'] == artist_name]['all_albums'].values[0]
    all_albums = all_albums[1:-1].split(',')
    all_albums = [int(album) for album in all_albums]

    if byName:
        all_albums = [convert_album_id_to_name(album) for album in all_albums]

    return all_albums

def get_all_artist_tracks(artist_name, byName=False):
    all_tracks = df_artist[df_artist['name'] == artist_name]['all_tracks'].values[0]
    all_tracks = all_tracks[1:-1].split(',')
    all_tracks = [int(track) for track in all_tracks]

    all_tracks = [convert_track_id_to_name(track) for track in all_tracks]

    return all_tracks

## Track Similarity

In [ ]:
# print all track ids that do not have a similarity value
temp_list = []
track_count = len(df_track)

track_ids = df_track['track_id'].to_list()
for track_id in track_ids:
    if df_similar[(df_similar['track_id_1'] == track_id) | (df_similar['track_id_2'] == track_id)].empty:
        temp_list.append(track_id)

print('Track ids without similarity value:', f"{temp_list[:10]} ..." if len(temp_list) > 10 else temp_list)
print('Number of track ids without similarity value:', len(temp_list))
print('Percentage of track ids without similarity value:', round(len(temp_list) / track_count * 100, 2), '%')

In [ ]:
df_similar.head(10)

In [ ]:
G = nx.Graph()
plt.figure(figsize=(30, 30), dpi=150)

# Add edges with weights
for _, row in df_similar.iterrows():
    t1 = row['track_name_1'].replace('$', '_')
    t2 = row['track_name_2'].replace('$', '_')
    G.add_edge(t1, t2, weight=row['sim_degree'])

# Find communities and assign colors
communities = community.greedy_modularity_communities(G)
node_colors = {}
for idx, comm in enumerate(communities):
    for node in comm:
        node_colors[node] = idx

pos = nx.spring_layout(G)

nx.draw_networkx_nodes(G, pos, node_size=300,
    node_color=[node_colors[node] for node in G.nodes()]
)

nx.draw_networkx_edges(G, pos, edgelist=G.edges(data=True),
    width=[edge[2]['weight'] * 5 for edge in G.edges(data=True)]
)

nx.draw_networkx_labels(G, pos, font_size=5, font_color="black")
plt.title("Song Similarity Graph")
plt.show()

In [ ]:
# Calculate the listeners of a track
def get_all_listeners(track_id):

    # filter the track
    track_data = df_track[df_track['track_id'] == track_id]
    track_data = track_data.reset_index(drop=True)

    # return the sum of the lastfm_playcount,lastfm_listeners 
    if track_data['lastfm_playcount'].values[0] != 'nan':
        ret1 = track_data['lastfm_listeners'].values[0]
    else: 
        ret1 = 0
    
    if track_data['lastfm_listeners'].values[0] != 'nan':
        ret2 = track_data['lastfm_playcount'].values[0]
    else:
        ret2 = 0

    return ret1 + ret2

# add a new column to the dataframe with the total number of listeners
df_track['total_listeners'] = df_track['track_id'].apply(get_all_listeners)
df_track.head()

In [ ]:
artist_total_listeners = df_track.groupby('artist_id').agg({'total_listeners': 'sum'}).reset_index()
artist_total_listeners['artist_name'] = artist_total_listeners['artist_id'].apply(convert_artist_id_to_name)
artist_total_listeners = artist_total_listeners.sort_values(by='total_listeners', ascending=False)

# filter out every artist with 0 listeners
print(artist_total_listeners[artist_total_listeners['total_listeners'] == 0].shape)
artist_total_listeners = artist_total_listeners[artist_total_listeners['total_listeners'] > 0]
print(artist_total_listeners.shape)

# sort the artists by the total number of listeners
artist_total_listeners = artist_total_listeners.sort_values(by='total_listeners', ascending=False)

artist_total_listeners.head()

## Intra-Artist Similarity

In [56]:
target_similar = df_similar[df_similar['artist_id_1'] == df_similar['artist_id_2']]

# Remove cases where only one track comparison exists
artist_counts = target_similar['artist_id_1'].value_counts()
valid_artists = artist_counts[artist_counts > 1].index
target_similar = target_similar[target_similar['artist_id_1'].isin(valid_artists)]

# Drop unnecessary columns and rename as needed
target_similar = target_similar.drop(columns=['artist_id_1', 'artist_id_2', 'track_id_1', 'track_id_2', 'artist_name_2'])
target_similar = target_similar.rename(columns={'artist_name_1': 'artist_name'})
target_similar = target_similar[['artist_name', 'track_name_1', 'track_name_2', 'sim_degree']]

# Average similarity degree for each artist and sort
target_similar = target_similar.groupby('artist_name').agg({'sim_degree': 'mean'}).reset_index()
target_similar = target_similar.sort_values(by='sim_degree', ascending=False)
target_similar = target_similar.reset_index(drop=True)

print(target_similar.shape)
target_similar.head(100)

(434, 2)


,artist_name,sim_degree
0,JoJo,1.000000
1,Ariel Pink,1.000000
2,The-Dream,1.000000
3,Autre Ne Veut,1.000000
4,Nelly Furtado,1.000000
...,...,...
95,Daft Punk,0.981978
96,Wavves,0.981926
97,Bucky Covington,0.981742
98,Phoenix,0.981470


In [57]:
# focus on the top 100 artists
top_artists_100 = artist_total_listeners.head(20)['artist_name'].to_list()
target_similar = target_similar[target_similar['artist_name'].isin(top_artists_100)]

print(target_similar.shape)
target_similar.head(20)

(18, 2)


,artist_name,sim_degree
53,Paramore,0.986908
84,Arcade Fire,0.983712
89,Vampire Weekend,0.983041
95,Daft Punk,0.981978
112,Mumford & Sons,0.980325
118,Lady Gaga,0.979587
119,Taylor Swift,0.979419
123,Bon Iver,0.978551
126,Coldplay,0.978178
135,Radiohead,0.977294


## Inter-Artist Similarity

In [ ]:
# group by artist_id1 and 2 and get the mean sim_degree
df_similar_artists = df_similar.groupby(['artist_name_1', 'artist_name_2']).agg({'sim_degree': 'mean'}).reset_index()
print(df_similar_artists.shape)
df_similar_artists.head()

In [ ]:
# remove all artists that have " and " or " & " or " featuring " in their name
artist_names = set(df_similar_artists['artist_name_1'].to_list() + df_similar_artists['artist_name_2'].to_list())

count = 0
for name in artist_names:
    if ' and ' in name or ' & ' in name or ' featuring ' in name:
        count += 1

print(f'Dropped {count} artists with " and ", " & " or " featuring " in their name')
df_similar_artists = df_similar_artists[~df_similar_artists['artist_name_1'].str.contains(' and | & | featuring ', case=False)]

In [ ]:
# remove artists that are not in the artist table (i.e. they have no listeners) 
artist_names = set(artist_total_listeners['artist_name'].to_list())

for row in df_similar_artists.iterrows():
    if row[1]['artist_name_1'] not in artist_names or row[1]['artist_name_2'] not in artist_names: # drop the row if both artists are not in the list
        df_similar_artists.drop(row[0], inplace=True)

print(df_similar_artists.shape)
df_similar_artists.head()

## Artist Popularity By Listener Count

In [ ]:
# get the listeners of a track
track_name = 'Blank Space'
track_id = convert_track_name_to_id(track_name)
listeners = get_all_listeners(track_id)
print(f'Total listeners of "{track_name}": {listeners}')


In [ ]:
G = nx.Graph()

for _, row in df_similar_artists.iterrows():
    G.add_edge(row['artist_name_1'], row['artist_name_2'], weight=row['sim_degree'])

# Set layout
plt.figure(figsize=(40, 40), dpi=200)
pos = nx.spring_layout(G, k=0.7, seed=42)
listeners_data = dict()

for row in artist_total_listeners.iterrows():
    listeners_data[row[1]['artist_name']] = row[1]['total_listeners']

max_listeners = max(listeners_data.values())
min_listeners = min(listeners_data.values())
norm = mcolors.Normalize(vmin=min_listeners, vmax=max_listeners)
cmap = cm.get_cmap("coolwarm")

node_colors = {
    artist: cmap(norm(listeners_data[artist]))
    for artist in G.nodes()
}

pos = nx.spring_layout(G, k=0.5, seed=42)
sorted_nodes = sorted(G.nodes(), key=lambda node: listeners_data[node])

for node in sorted_nodes:
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_size=[5000],
        node_color=[node_colors[node]]
    )

nx.draw_networkx_edges(G, pos, edgelist=G.edges(data=True), width= 0.1)
nx.draw_networkx_labels(G, pos, font_size=4, font_color="black")
plt.title("Artist Similarity Graph with Listener-Based Coloring")
plt.show()

In [17]:
top_artists = artist_total_listeners.head(20)['artist_name'].to_list()
print (artist_total_listeners.shape)
artist_total_listeners.head(20)

(1052, 3)


,artist_id,total_listeners,artist_name
2191,2192,123022947,The Beatles
150,151,120455219,Coldplay
297,298,117662840,Lady Gaga
1517,1518,114602377,Radiohead
433,434,113920714,Arcade Fire
1474,1475,110526377,Death Cab For Cutie
242,243,105486282,Lana Del Rey
655,656,104982299,The Killers
2258,2259,101311518,The xx
627,628,99191515,Linkin Park


## Find Unpopolar Artists Similar to Very Popular Ones 

In [39]:
# Find Unpopolar Artists Similar to Very Popular Ones

# Top Artists by Listener Count
top_artists = artist_total_listeners.head(20)['artist_name'].to_list()
top_artists_100 = artist_total_listeners.head(100)['artist_name']

# Unpopular artists are the ones that are not in the top 100
unpopular_artists = artist_total_listeners[~artist_total_listeners['artist_name'].isin(top_artists_100)]['artist_name'].to_list()

# Find unpopular artists that are similar to popular ones
df_similar_artists_top = df_similar_artists[df_similar_artists['artist_name_1'].isin(top_artists) | df_similar_artists['artist_name_2'].isin(top_artists)]
df_similar_artists_top = df_similar_artists_top[df_similar_artists_top['sim_degree'] > 0.5]
df_similar_artists_top = df_similar_artists_top[df_similar_artists_top['artist_name_1'] != df_similar_artists_top['artist_name_2']]

# Include their listeners
df_similar_artists_top = df_similar_artists_top.merge(artist_total_listeners, left_on='artist_name_1', right_on='artist_name')
df_similar_artists_top = df_similar_artists_top.rename(columns={'total_listeners': 'listeners_1'})
df_similar_artists_top = df_similar_artists_top.drop(columns=['artist_name', 'artist_id'])
df_similar_artists_top = df_similar_artists_top.merge(artist_total_listeners, left_on='artist_name_2', right_on='artist_name')
df_similar_artists_top = df_similar_artists_top.rename(columns={'total_listeners': 'listeners_2'})
df_similar_artists_top = df_similar_artists_top.drop(columns=['artist_name', 'artist_id'])

# Rescale the listeners to be between 0 and 1
df_similar_artists_top['listeners_1'] = round(df_similar_artists_top['listeners_1'] / max_listeners, 2)
df_similar_artists_top['listeners_2'] = round(df_similar_artists_top['listeners_2'] / max_listeners, 2)
df_similar_artists_top['sim_degree'] = round(df_similar_artists_top['sim_degree'] , 2)

# Rorder columns
df_similar_artists_top = df_similar_artists_top[['artist_name_1', 'listeners_1', 'artist_name_2', 'listeners_2', 'sim_degree']]

print(df_similar_artists_top.shape)
df_similar_artists_top.head(20)

(12, 5)


,artist_name_1,listeners_1,artist_name_2,listeners_2,sim_degree
0,Arcade Fire,0.93,Muse,0.80,0.67
1,Becky G,0.01,Taylor Swift,0.65,0.56
2,Bonnie McKee,0.00,Katy Perry,0.56,0.95
3,Bonnie McKee,0.00,Lady Gaga,0.96,0.70
4,Katy Perry,0.56,Pitbull,0.04,0.89
5,Meghan Trainor,0.02,Taylor Swift,0.65,0.54
6,Passion Pit,0.26,Arcade Fire,0.93,0.72
7,Rita Ora,0.01,Katy Perry,0.56,0.56
8,Rita Ora,0.01,Taylor Swift,0.65,0.58
9,Sam Smith,0.02,Coldplay,0.98,0.51
